## 问题定义

本问题为求解固液体系中的传热。其大致定义参考下图：

![problem](./resource/problem.png)


其中中间的小物体为固体，外围为低温流体。

具体参数如下：

### 流体参数
* 流体运动粘度：$0.004195088 m^2/s$
* 流体密度：$1.1614 kg/m^3$
* 流体比热：$1005 J/(kg \cdot K)$
* 流体热导率：$0.0261 W / (m \cdot K)$

### 固体参数
* 固体密度：$8930 kg/m^3$
* 固体比热：$385 J/(kg \cdot K)$
* 固体热导率：$385 W / (m \cdot K)$

### 环境参数
* 入口流体速度：$5.24386 m/s$
* 入口温度：$25 ℃$
* 固体热源热流：$51.948051948 W / m^2$

### 无量纲因子
* 长度尺度 (length_scale)：$0.04 m$
* 时间尺度 (time_scale)：$0.007627968710072352 s$
* 质量尺度 (mass_scale)：$7.43296e-05 kg$
* 温度尺度 (temp_scale)：$1 K$
* 速度尺度 (velocity_scale)：length_scale / time_scale $m/s$
* 压力尺度 (pressure_scale)：mass_scale / (length_scale * time_scale^2) $ kg / (m \cdot s^2)$
* 密度尺度 (density_scale)：mass_scale / length_scale^3 $kg/m^3$
* 功率尺度 (watt_scale)：(mass_scale * length_scale^2) / (time_scale^3) $kg m^2 / s^3$
* 力尺度（joule_scale）：(mass_scale * length_scale^2) / (time_scale^2) $kg m^2 / s^2$

### 几何
* channel:
    * 长：-2.5, 5.0
    * 宽：-0.5, 0.5
* chip:
    * 长：-1.0, 0.0
    * 宽：-0.5, 0.1
* 热源：
    * 长：-0.7, -0.5
    * 宽：-0.5，-0.5


### 求解目标

要求求解上述求解区域的稳态解

## 基本描述

在本例中流动和传热是单向耦合。具体而言，流动影响传热，但是传热不影响流动。因此分两步求解，首先求解流动，然后将流动的模型固定并求解传热。

本文件为求解heat transfer的代码，求解flow请参考16_2D_ChipSolidFluid_Flow。

注意，需要提前copy 中的flow_network.0.pth到本工程目录下的outputs文件夹中


## 求解flow

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

In [3]:
import torch
import numpy as np
from sympy import Symbol, Eq, tanh, Or, And

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.sym.utils.sympy.functions import parabola
from modulus.sym.eq.pdes.advection_diffusion import AdvectionDiffusion
from modulus.sym.eq.pdes.navier_stokes import GradNormal
from modulus.sym.eq.pdes.diffusion import Diffusion, DiffusionInterface
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)
from modulus.sym.domain.monitor import PointwiseMonitor
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.models.fourier_net import FourierNetArch
from modulus.sym.models.modified_fourier_net import ModifiedFourierNetArch

In [4]:
cfg = compose(config_path="conf", config_name="config_heat")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 150000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 1000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch

### 定义必要组件

#### Geo

In [5]:
#############
# Real Params
#############
fluid_kinematic_viscosity = 0.004195088  # m**2/s
fluid_density = 1.1614  # kg/m**3
fluid_specific_heat = 1005  # J/(kg K)
fluid_conductivity = 0.0261  # W/(m K)

# copper params
copper_density = 8930  # kg/m3
copper_specific_heat = 385  # J/(kg K)
copper_conductivity = 385  # W/(m K)

# boundary params
inlet_velocity = 5.24386  # m/s
inlet_temp = 25.0  # C
copper_heat_flux = 51.948051948  # W / m2

################
# Non dim params
################
length_scale = 0.04  # m
time_scale = 0.007627968710072352  # s
mass_scale = 7.43296e-05  # kg
temp_scale = 1.0  # K
velocity_scale = length_scale / time_scale  # m/s
pressure_scale = mass_scale / (length_scale * time_scale**2)  # kg / (m s**2)
density_scale = mass_scale / length_scale**3  # kg/m3
watt_scale = (mass_scale * length_scale**2) / (time_scale**3)  # kg m**2 / s**3
joule_scale = (mass_scale * length_scale**2) / (
    time_scale**2
)  # kg * m**2 / s**2

##############################
# Nondimensionalization Params
##############################
# fluid params
nd_fluid_kinematic_viscosity = fluid_kinematic_viscosity / (
    length_scale**2 / time_scale
)
nd_fluid_density = fluid_density / density_scale
nd_fluid_specific_heat = fluid_specific_heat / (
    joule_scale / (mass_scale * temp_scale)
)
nd_fluid_conductivity = fluid_conductivity / (
    watt_scale / (length_scale * temp_scale)
)
nd_fluid_diffusivity = nd_fluid_conductivity / (
    nd_fluid_specific_heat * nd_fluid_density
)

# copper params
nd_copper_density = copper_density / (mass_scale / length_scale**3)
nd_copper_specific_heat = copper_specific_heat / (
    joule_scale / (mass_scale * temp_scale)
)
nd_copper_conductivity = copper_conductivity / (
    watt_scale / (length_scale * temp_scale)
)
nd_copper_diffusivity = nd_copper_conductivity / (
    nd_copper_specific_heat * nd_copper_density
)

# boundary params
nd_inlet_velocity = inlet_velocity / (length_scale / time_scale)
nd_inlet_temp = inlet_temp / temp_scale
nd_copper_source_grad = copper_heat_flux * length_scale / temp_scale

In [6]:
# add constraints to solver
# simulation params
channel_length = (-2.5, 5.0)
channel_width = (-0.5, 0.5)
chip_pos = -1.0
chip_height = 0.6
chip_width = 1.0

source_origin = (-0.7, -0.5)
source_dim = (0.4, 0.0)
source_length = 0.4

# define sympy variables to parametrize domain curves
x, y = Symbol("x"), Symbol("y")

# define geometry
channel = Channel2D(
    (channel_length[0], channel_width[0]), (channel_length[1], channel_width[1])
)  # channel
inlet = Line(
    (channel_length[0], channel_width[0]),
    (channel_length[0], channel_width[1]),
    normal=1,
)  # 入口
outlet = Line(
    (channel_length[1], channel_width[0]),
    (channel_length[1], channel_width[1]),
    normal=1,
)  # 出口
rec = Rectangle(
    (chip_pos, channel_width[0]),
    (chip_pos + chip_width, channel_width[0] + chip_height),
)  # chip

chip2d = rec
geo = channel - rec  # 流体区域
x_pos = Symbol("x_pos")
integral_line = Line((x_pos, channel_width[0]), (x_pos, channel_width[1]), 1)
x_pos_range = {
    x_pos: lambda batch_size: np.full(
        (batch_size, 1), np.random.uniform(channel_length[0], channel_length[1])
    )
}

#### PDE

In [7]:
ad = AdvectionDiffusion(
    T="theta_f", rho=nd_fluid_density, D=nd_fluid_diffusivity, dim=2, time=False
)  # 平流扩散方程
diff = Diffusion(T="theta_s", D=1.0, dim=2, time=False)  # 固体内部扩散
interface = DiffusionInterface(
    "theta_f",
    "theta_s",
    nd_fluid_conductivity,
    nd_copper_conductivity,
    dim=2,
    time=False,
) # 界面
gn_theta_f = GradNormal("theta_f", dim=2, time=False)  # Neumann边界
gn_theta_s = GradNormal("theta_s", dim=2, time=False)  # Neumann边界

#### Model

In [8]:
# 定义为FourierNet
# 输入为空间坐标
# 输出为预测的速度和压强
flow_net = FourierNetArch(
        input_keys=[Key("x"), Key("y")],
        output_keys=[Key("u"), Key("v"), Key("p")],
        frequencies=("axis", [i / 5.0 for i in range(25)]),
)  # 流动模型（需要与flow解算的保持一致）
solid_heat_net = ModifiedFourierNetArch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("theta_s_star")],
    layer_size=256,
    frequencies=("gaussian", 2, 128),
)  # 固体传热模型
fluid_heat_net = ModifiedFourierNetArch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("theta_f_star")],
    layer_size=256,
    frequencies=("gaussian", 2, 128),
)  # 流体传热模型

#### Node

In [9]:
nodes = (
    ad.make_nodes(detach_names=["u", "v"])  # 平流扩散，注意传入的是截断的速度
    + diff.make_nodes()  # 固体内部扩散
    + interface.make_nodes()  # 界面
    + gn_theta_f.make_nodes()  # Neumann边界
    + gn_theta_s.make_nodes()  # Neumann边界
    + [
        Node.from_sympy(Symbol("theta_s_star") + 170.0, "theta_s")
    ]  # Normalize the outputs
    + [
        Node.from_sympy(Symbol("theta_f_star") + 70.0, "theta_f")
    ]  # Normalize the outputs
    + [flow_net.make_node(name="flow_network", optimize=False)]  # 流体模型，注意optimize=False，直接加载
    + [solid_heat_net.make_node(name="solid_heat_network")]
    + [fluid_heat_net.make_node(name="fluid_heat_network")]
)

#### Domain

In [10]:
domain = Domain()

#### 入口条件

In [11]:
inlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=inlet,
    outvar={"theta_f": nd_inlet_temp},
    batch_size=cfg.batch_size.inlet,
    lambda_weighting={"theta_f": 100.0},
)
domain.add_constraint(inlet, "inlet")  # 入口温度约束

#### 出口条件

In [12]:
outlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=outlet,
    outvar={"normal_gradient_theta_f": 0},
    batch_size=cfg.batch_size.outlet,
    criteria=Eq(x, channel_length[1]),
)
domain.add_constraint(outlet, "outlet")  # 出口热梯度约束

#### channel的绝缘边缘

In [13]:
def walls_criteria(invar, params):
    sdf = chip2d.sdf(invar, params)
    return np.less(sdf["sdf"], -1e-5)

walls = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=channel,
    outvar={"normal_gradient_theta_f": 0},
    batch_size=cfg.batch_size.walls,
    criteria=walls_criteria,
)
domain.add_constraint(walls, "channel_walls")

#### 平流扩散 （远离chip的区域）

In [14]:
interior_lr = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"advection_diffusion_theta_f": 0},
    batch_size=cfg.batch_size.interior_lr,
    criteria=Or(x < (chip_pos - 0.25), x > (chip_pos + chip_width + 0.25)),
    lambda_weighting={"advection_diffusion_theta_f": 1.0},
)
domain.add_constraint(interior_lr, "fluid_interior_lr")

#### 平流扩散 （靠近chip的区域）

In [15]:
interior_hr = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"advection_diffusion_theta_f": 0},
    batch_size=cfg.batch_size.interior_hr,
    criteria=And(x > (chip_pos - 0.25), x < (chip_pos + chip_width + 0.25)),
    lambda_weighting={"advection_diffusion_theta_f": 1.0},
)
domain.add_constraint(interior_hr, "fluid_interior_hr")

#### 固体内部扩散

In [16]:
interiorS = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=chip2d,
    outvar={"diffusion_theta_s": 0},
    batch_size=cfg.batch_size.interiorS,
    lambda_weighting={"diffusion_theta_s": 1.0},
)
domain.add_constraint(interiorS, "solid_interior")

#### 界面

In [17]:
def interface_criteria(invar, params):
    sdf = channel.sdf(invar, params)
    return np.greater(sdf["sdf"], 0)

interface = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=chip2d,
    outvar={
        "diffusion_interface_dirichlet_theta_f_theta_s": 0,
        "diffusion_interface_neumann_theta_f_theta_s": 0,
    },
    batch_size=cfg.batch_size.interface,
    lambda_weighting={
        "diffusion_interface_dirichlet_theta_f_theta_s": 1,
        "diffusion_interface_neumann_theta_f_theta_s": 1e-4,
    },
    criteria=interface_criteria,
)
domain.add_constraint(interface, name="interface")

#### 热源约束

In [18]:
heat_source = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=chip2d,
    outvar={"normal_gradient_theta_s": nd_copper_source_grad},
    batch_size=cfg.batch_size.heat_source,
    lambda_weighting={"normal_gradient_theta_s": 100},
    criteria=(
        Eq(y, source_origin[1])
        & (x >= source_origin[0])
        & (x <= (source_origin[0] + source_dim[0]))
    ),
)
domain.add_constraint(heat_source, name="heat_source")

#### chip边界绝缘约束

In [19]:
chip_walls = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=chip2d,
    outvar={"normal_gradient_theta_s": 0},
    batch_size=cfg.batch_size.chip_walls,
    criteria=(
        Eq(y, source_origin[1])
        & ((x < source_origin[0]) | (x > (source_origin[0] + source_dim[0])))
    ),
)
domain.add_constraint(chip_walls, name="chip_walls")

#### 验证器以及其他必要组件

In [20]:
monitor = PointwiseMonitor(
    chip2d.sample_boundary(10000, criteria=Eq(y, source_origin[1])),
    output_names=["theta_s"],
    metrics={
        "peak_temp": lambda var: torch.max(var["theta_s"]),
    },
    nodes=nodes,
)
domain.add_monitor(monitor)  # 峰值温度

In [21]:
# 流体传热
file_path = "openfoam/2d_real_cht_fluid.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {
        "x-coordinate": "x",
        "y-coordinate": "y",
        "x-velocity": "u",
        "y-velocity": "v",
        "pressure": "p",
        "temperature": "theta_f",
    }
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] = openfoam_var["x"] / length_scale - 2.5  # normalize pos
    openfoam_var["y"] = openfoam_var["y"] / length_scale - 0.5
    openfoam_var["p"] = (openfoam_var["p"] + 400.0) / pressure_scale
    openfoam_var["u"] = openfoam_var["u"] / velocity_scale
    openfoam_var["v"] = openfoam_var["v"] / velocity_scale
    openfoam_var["theta_f"] = openfoam_var["theta_f"] - 273.15

    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value
        for key, value in openfoam_var.items()
        if key in ["u", "v", "p", "theta_f"]
    }
    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

In [22]:
# 固体传热

file_path = "openfoam/2d_real_cht_solid.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"x-coordinate": "x", "y-coordinate": "y", "temperature": "theta_s"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] = openfoam_var["x"] / length_scale - 2.5  # normalize pos
    openfoam_var["y"] = openfoam_var["y"] / length_scale - 0.5
    openfoam_var["theta_s"] = openfoam_var["theta_s"] - 273.15

    openfoam_solid_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_solid_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["theta_s"]
    }
    openfoam_solid_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_solid_invar_numpy,
        true_outvar=openfoam_solid_outvar_numpy,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_solid_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [23]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [24]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [25]:
slv.solve()

2024-03-01 12:24:03,443 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-01 12:24:03,453 - INFO - attempting to restore from: /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 12:24:03,474 - INFO - Success loading optimizer: /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs/optim_checkpoint.0.pth
2024-03-01 12:24:03,481 - INFO - Success loading model: /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs/flow_network.0.pth
2024-03-01 12:24:03,487 - INFO - Success loading model: /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs/fluid_heat_network.0.pth
2024-03-01 12:24:03,493 - INFO - Success loading model: /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs/solid_heat_network.0.pth
2024-03-01 12:24:05,194 - INFO - [step:     104000] record constraint batch time:  1.771e-01s
2024-03-01 12:24:19,883 - INFO - [step:     104000] record validators time:  1.469e+01s
2024

2024-03-01 12:30:14,958 - INFO - [step:     109000] loss:  1.013e+01, time/iteration:  2.126e+02 ms
2024-03-01 12:30:20,426 - INFO - [step:     109100] loss:  8.012e+00, time/iteration:  5.467e+01 ms
2024-03-01 12:30:25,812 - INFO - [step:     109200] loss:  1.258e+01, time/iteration:  5.384e+01 ms
2024-03-01 12:30:31,202 - INFO - [step:     109300] loss:  9.878e+00, time/iteration:  5.390e+01 ms
2024-03-01 12:30:36,463 - INFO - [step:     109400] loss:  8.937e+00, time/iteration:  5.259e+01 ms
2024-03-01 12:30:41,715 - INFO - [step:     109500] loss:  9.338e+00, time/iteration:  5.249e+01 ms
2024-03-01 12:30:47,191 - INFO - [step:     109600] loss:  7.102e+00, time/iteration:  5.475e+01 ms
2024-03-01 12:30:52,429 - INFO - [step:     109700] loss:  7.839e+00, time/iteration:  5.236e+01 ms
2024-03-01 12:30:57,652 - INFO - [step:     109800] loss:  7.476e+00, time/iteration:  5.220e+01 ms
2024-03-01 12:31:03,048 - INFO - [step:     109900] loss:  9.074e+00, time/iteration:  5.395e+01 ms


2024-03-01 12:37:00,278 - INFO - [step:     115000] saved checkpoint to /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 12:37:00,281 - INFO - [step:     115000] loss:  1.307e+01, time/iteration:  1.912e+02 ms
2024-03-01 12:37:05,509 - INFO - [step:     115100] loss:  2.265e+01, time/iteration:  5.227e+01 ms
2024-03-01 12:37:10,734 - INFO - [step:     115200] loss:  9.042e+00, time/iteration:  5.224e+01 ms
2024-03-01 12:37:15,946 - INFO - [step:     115300] loss:  7.283e+00, time/iteration:  5.211e+01 ms
2024-03-01 12:37:21,129 - INFO - [step:     115400] loss:  1.251e+01, time/iteration:  5.182e+01 ms
2024-03-01 12:37:26,339 - INFO - [step:     115500] loss:  1.633e+01, time/iteration:  5.208e+01 ms
2024-03-01 12:37:31,574 - INFO - [step:     115600] loss:  1.381e+01, time/iteration:  5.234e+01 ms
2024-03-01 12:37:36,807 - INFO - [step:     115700] loss:  9.397e+00, time/iteration:  5.232e+01 ms
2024-03-01 12:37:41,998 - INFO - [step:     115800] loss:  1.174e+01, time/

2024-03-01 12:43:44,494 - INFO - [step:     121000] record monitor time:  3.713e-03s
2024-03-01 12:43:44,533 - INFO - [step:     121000] saved checkpoint to /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 12:43:44,535 - INFO - [step:     121000] loss:  6.592e+00, time/iteration:  1.940e+02 ms
2024-03-01 12:43:49,803 - INFO - [step:     121100] loss:  8.688e+00, time/iteration:  5.267e+01 ms
2024-03-01 12:43:55,023 - INFO - [step:     121200] loss:  8.549e+00, time/iteration:  5.219e+01 ms
2024-03-01 12:44:00,304 - INFO - [step:     121300] loss:  7.328e+00, time/iteration:  5.278e+01 ms
2024-03-01 12:44:05,484 - INFO - [step:     121400] loss:  7.558e+00, time/iteration:  5.178e+01 ms
2024-03-01 12:44:10,695 - INFO - [step:     121500] loss:  7.726e+00, time/iteration:  5.210e+01 ms
2024-03-01 12:44:15,900 - INFO - [step:     121600] loss:  8.112e+00, time/iteration:  5.202e+01 ms
2024-03-01 12:44:21,103 - INFO - [step:     121700] loss:  1.587e+01, time/iteration:  5.2

2024-03-01 12:50:29,881 - INFO - [step:     127000] record validators time:  1.295e+01s
2024-03-01 12:50:29,904 - INFO - [step:     127000] record monitor time:  4.854e-03s
2024-03-01 12:50:29,951 - INFO - [step:     127000] saved checkpoint to /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 12:50:29,953 - INFO - [step:     127000] loss:  1.653e+01, time/iteration:  1.989e+02 ms
2024-03-01 12:50:35,187 - INFO - [step:     127100] loss:  8.202e+00, time/iteration:  5.234e+01 ms
2024-03-01 12:50:40,426 - INFO - [step:     127200] loss:  7.968e+00, time/iteration:  5.237e+01 ms
2024-03-01 12:50:45,667 - INFO - [step:     127300] loss:  7.449e+00, time/iteration:  5.239e+01 ms
2024-03-01 12:50:51,028 - INFO - [step:     127400] loss:  9.010e+00, time/iteration:  5.360e+01 ms
2024-03-01 12:50:56,320 - INFO - [step:     127500] loss:  1.635e+01, time/iteration:  5.290e+01 ms
2024-03-01 12:51:01,740 - INFO - [step:     127600] loss:  8.938e+00, time/iteration:  5.417e+01 ms
20

2024-03-01 12:57:03,035 - INFO - [step:     133000] record constraint batch time:  2.245e-01s
2024-03-01 12:57:16,556 - INFO - [step:     133000] record validators time:  1.352e+01s
2024-03-01 12:57:16,588 - INFO - [step:     133000] record monitor time:  1.347e-02s
2024-03-01 12:57:16,634 - INFO - [step:     133000] saved checkpoint to /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 12:57:16,636 - INFO - [step:     133000] loss:  1.350e+01, time/iteration:  2.025e+02 ms
2024-03-01 12:57:21,846 - INFO - [step:     133100] loss:  6.913e+00, time/iteration:  5.209e+01 ms
2024-03-01 12:57:27,084 - INFO - [step:     133200] loss:  7.924e+00, time/iteration:  5.236e+01 ms
2024-03-01 12:57:32,491 - INFO - [step:     133300] loss:  9.122e+00, time/iteration:  5.405e+01 ms
2024-03-01 12:57:37,738 - INFO - [step:     133400] loss:  8.048e+00, time/iteration:  5.246e+01 ms
2024-03-01 12:57:42,942 - INFO - [step:     133500] loss:  2.686e+01, time/iteration:  5.202e+01 ms
2024-03-

2024-03-01 13:04:47,169 - INFO - [step:     138900] loss:  7.220e+00, time/iteration:  9.358e+01 ms
2024-03-01 13:04:57,262 - INFO - [step:     139000] record constraint batch time:  3.752e-01s
2024-03-01 13:05:13,633 - INFO - [step:     139000] record validators time:  1.637e+01s
2024-03-01 13:05:13,720 - INFO - [step:     139000] record monitor time:  5.121e-02s
2024-03-01 13:05:13,788 - INFO - [step:     139000] saved checkpoint to /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 13:05:13,790 - INFO - [step:     139000] loss:  8.022e+00, time/iteration:  2.662e+02 ms
2024-03-01 13:05:23,825 - INFO - [step:     139100] loss:  9.647e+00, time/iteration:  1.003e+02 ms
2024-03-01 13:05:33,806 - INFO - [step:     139200] loss:  1.026e+01, time/iteration:  9.979e+01 ms
2024-03-01 13:05:43,820 - INFO - [step:     139300] loss:  8.942e+00, time/iteration:  1.001e+02 ms
2024-03-01 13:05:53,721 - INFO - [step:     139400] loss:  7.274e+00, time/iteration:  9.900e+01 ms
2024-03-

2024-03-01 13:15:52,258 - INFO - [step:     144800] loss:  8.089e+00, time/iteration:  9.486e+01 ms
2024-03-01 13:16:01,690 - INFO - [step:     144900] loss:  7.555e+00, time/iteration:  9.426e+01 ms
2024-03-01 13:16:13,290 - INFO - [step:     145000] record constraint batch time:  3.750e-01s
2024-03-01 13:16:29,181 - INFO - [step:     145000] record validators time:  1.589e+01s
2024-03-01 13:16:29,232 - INFO - [step:     145000] record monitor time:  9.619e-03s
2024-03-01 13:16:29,286 - INFO - [step:     145000] saved checkpoint to /workspace/17_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 13:16:29,287 - INFO - [step:     145000] loss:  8.625e+00, time/iteration:  2.760e+02 ms
2024-03-01 13:16:39,332 - INFO - [step:     145100] loss:  8.106e+00, time/iteration:  1.004e+02 ms
2024-03-01 13:16:49,164 - INFO - [step:     145200] loss:  7.410e+00, time/iteration:  9.829e+01 ms
2024-03-01 13:16:56,982 - INFO - [step:     145300] loss:  8.784e+00, time/iteration:  7.815e+01 ms
2024-03-

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![solid temp](./outputs/validators/validator_2_theta_s.png)

![fluid temp](./outputs/validators/validator_theta_f.png)

![u](./outputs/validators/validator_u.png)

![v](./outputs/validators/validator_v.png)

![p](./outputs/validators/validator_p.png)